In [40]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()
print(len(documents_raw))
documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

3


In [41]:
len(documents)

948

# Question 3. Searching: score for top record (1 point)

# Question 4. Filtering: 3rd document (1 point)

In [42]:
# Question 5. Prompt length (1 point)

In [43]:
# Question 6. Number of tokens (1 point)

In [44]:
query ="How do I execute a command in a running docker container?"

In [45]:
from elasticsearch import Elasticsearch
es_client=Elasticsearch('http://localhost:9200')
index_settings={
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}
index_name='course-questions'
es_client.indices.create(index=index_name, body=index_settings)

BadRequestError: BadRequestError(400, 'resource_already_exists_exception', 'index [course-questions/bRKgAPSETDqwNIZcPX7mBw] already exists')

In [46]:
from tqdm.auto import tqdm
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 948/948 [00:25<00:00, 36.95it/s]


In [57]:
query = "How do I execute a command in a running docker container?"
search_query={
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                }
            }
        }
    }
response=es_client.search(index=index_name, body=search_query)

In [58]:
# score of the top items
response['hits']['hits'][0]['_score']         # expecting 84.050095

84.24209

In [66]:
query = "How do I execute a command in a running docker container?"
search_query={
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }
response=es_client.search(index=index_name, body=search_query)

In [80]:
# the third document
response['hits']['hits'][2]   # How do I copy files from a different folder into docker container’s working directory?

{'_index': 'course-questions',
 '_id': 'UJi0f5ABJ-e-nKIe7dsM',
 '_score': 84.25498,
 '_source': {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I debug a docker container?',
  'course': 'machine-learning-zoomcamp'}}

In [74]:
context_template = """
Q: {question}
A: {text}
""".strip()

prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

In [77]:
context_pieces = []

for hit in response['hits']['hits']:
    doc = hit['_source']
    context_piece = context_template.format(**doc)
    context_pieces.append(context_piece)

context = '\n\n'.join(context_pieces)

In [81]:
prompt = prompt_template.format(question=query, context=context)
len(prompt)   # 1462 expected

2042

In [86]:
!pip install tiktoken

In [91]:
import tiktoken

ModuleNotFoundError: No module named 'tiktoken'

In [88]:
print(prompt[:100])


You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.



In [89]:
encoding = tiktoken.encoding_for_model("gpt-4o")
len(encoding.encode(prompt))

NameError: name 'tiktoken' is not defined

In [73]:
def elastic_search(query):
    search_query={
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                }
               
            }
        }
    }
    response=es_client.search(index=index_name, body=search_query)
    result_doc=[]
    for hit in response['hits']['hits']:
        result_doc.append(hit['_source'])
    return result_doc

In [50]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [51]:
def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [24]:
result=elastic_search(query)
result[0]

{'text': 'In case running pgcli  locally causes issues or you do not want to install it locally you can use it running in a Docker container instead.\nBelow the usage with values used in the videos of the course for:\nnetwork name (docker network)\npostgres related variables for pgcli\nHostname\nUsername\nPort\nDatabase name\n$ docker run -it --rm --network pg-network ai2ys/dockerized-pgcli:4.0.1\n175dd47cda07:/# pgcli -h pg-database -U root -p 5432 -d ny_taxi\nPassword for root:\nServer: PostgreSQL 16.1 (Debian 16.1-1.pgdg120+1)\nVersion: 4.0.1\nHome: http://pgcli.com\nroot@pg-database:ny_taxi> \\dt\n+--------+------------------+-------+-------+\n| Schema | Name             | Type  | Owner |\n|--------+------------------+-------+-------|\n| public | yellow_taxi_data | table | root  |\n+--------+------------------+-------+-------+\nSELECT 1\nTime: 0.009s\nroot@pg-database:ny_taxi>',
 'section': 'Module 1: Docker and Terraform',
 'question': 'PGCLI - running in a Docker container',
 'co

In [18]:
from openai import OpenAI
client = OpenAI()
def rag(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [19]:
rag(query)

'To execute a command in a running Docker container, you need to use the `docker exec` command. Unfortunately, the provided context does not include information about the `docker exec` command explicitly. \n\nHowever, based on typical usage, you can execute a command in a running Docker container with the following syntax:\n\n```sh\ndocker exec -it <container_id_or_name> <command>\n```\n\nFor example, if you want to execute a shell in a running container, you would use:\n\n```sh\ndocker exec -it <container_id_or_name> /bin/bash\n```\n\nTo list tables using `pgcli` in a PostgreSQL container (as depicted in the provided example), you could first ensure your container is running with the required network and environment variables and then run the following command directly in the container:\n\n```sh\ndocker exec -it <container_id_or_name> pgcli -h pg-database -U root -p 5432 -d ny_taxi\n```\n\nReplace `<container_id_or_name>` with the actual ID or name of the running container.'